In [1]:
'''
Date: 2022.07.29
Title: 데이터 분류
By: Kang Jin Seong
'''
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

'''
감성 분류: 문서(텍스트 데이터)를 긍정의견 또는 부정의견으로 나누어 분류하는것

'''

'\n감성 분류: 문서(텍스트 데이터)를 긍정의견 또는 부정의견으로 나누어 분류하는것\n\n'

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

# 윈도우용 크롬 웹드라이버 실행 경로
excutable_path = 'C:/Users/USER/DataAnalysis_jupyter/chromedriver.exe'

#크롤링할 사이트 주소를 정의합니다.
source_url = 'https://map.kakao.com/'

#크롬 드라이버를 사용합니다.
driver = webdriver.Chrome(executable_path = excutable_path)

#카카오 지도에 접속합니다.
driver.get(source_url)

# 검색창에 검색어를 입력합니다.
searchbox = driver.find_element('xpath',"//input[@id= 'search.keyword.query']")
searchbox.send_keys('서울 맛집')

# 검색버튼을 눌러서 결과를 가져옵니다.
searchbutton = driver.find_element('xpath', "//button[@id = 'search.keyword.submit']")
driver.execute_script('arguments[0].click();', searchbutton)

# 검색 결과를 가져올 시간을 기다립니다.
time.sleep(2)

# 검색 결과의 페이지 소스를 가져옵니다.
html = driver.page_source

# BeautifulSoup을 이용하여 html 정보를 파싱합니다.
soup = BeautifulSoup(html, 'html.parser')
moreviews = soup.find_all(name = 'a', attrs = {'class':'moreview'})

# a 태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []

for moreview in moreviews:
    page_url = moreview.get('href')
    print(page_url)
    page_urls.append(page_url)
    
# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://place.map.kakao.com/9314709
https://place.map.kakao.com/1974266862
https://place.map.kakao.com/10332413
https://place.map.kakao.com/1065693087
https://place.map.kakao.com/348276052
https://place.map.kakao.com/27531028
https://place.map.kakao.com/13559837
https://place.map.kakao.com/1950123074
https://place.map.kakao.com/1877580394
https://place.map.kakao.com/220597413
https://place.map.kakao.com/1770731230
https://place.map.kakao.com/8025375
https://place.map.kakao.com/27584230
https://place.map.kakao.com/27540102
https://place.map.kakao.com/353512935
https://place.map.kakao.com/1492599844


In [3]:
'''
가져온 맛집 리스트의 리뷰 정보 크롤링하기
'''

columns = ['score', 'review']
df = pd.DataFrame(columns = columns)

driver = webdriver.Chrome(executable_path = excutable_path)

for page_url in page_urls:
    
    # 상세 보기 페이지에 접속합니다.
    driver.get(page_url)
    time.sleep(2)
    
    # 첫 페이지 리뷰를 크롤링합니다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents_div = soup.find(name = 'div', attrs = {'class':'evaluation_review'})
    
    # 별점을 가져옵니다.
    try:
        rates = contents_div.find_all(name = 'em', attrs = {"class": "num_rate"})
    except:
        break
    # 리뷰를 가져옵니다.
    reviews = contents_div.find_all(name = 'p', attrs = {"class": "txt_comment"})
    
    for rate, review in zip(rates, reviews):
        row = [rate.text[0], review.find(name = 'span').text]
        series = pd.Series(row, index = df.columns)
        df = df.append(series, ignore_index = True)
        
    # 2-5페이지의 리뷰를 크롤링합니다.
    for button_num in range(2,6):
        # 오류가 나는 경우 (리뷰 페이지가 없는 경우) 수행하지 않습니다.
        
        try:
            another_reviews = driver.find_element('xpath', "//a[@data-page  = '" + str(button_num) + "']")
            another_reviews.click()
            time.sleep(2)
            
            # 페이지 리뷰를 크롱링합니다.
            
            html = driverr.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents_div = soup.find(name = 'div', attrs = {"class": "evaluation_review"})
            
            # 별점을 가져옵니다.
            rates = contents_div.find_all(name = 'em', attrs = {"class": "num_rate"})
            
            # 리뷰를 가져옵니다.
            reviews = contents_div.find_all(name = 'p', attrs = {"class": "txt_comment"})
            
            for rate, review in zip(rates, reviews):
                row = [rate.text[0], review.find(name = 'span').text]
                series = pd.Series(row, index = df.columns)
                df = df.append(series, ignore_index = True)
                
        except:
            break
driver.close()

In [5]:
# 4점 이상의 리뷰는 긍정, 3점 이하의 리뷰는 부정으로 평가합니다.
df['y'] = df['score'].apply(lambda x: 1 if float(x) > 3 else 0)
print(df.shape)
df.head()

(80, 3)


,score,review,y
0,3,맛은 있는데 가격이 아쉽 !,0
1,4,전 후기 잘 안쓰는게 실망했다는 리뷰가 많아 써봅니다. 다른메뉴는 모르겠는데 이집은...,1
2,2,대실망입니다 양녕갈비살 정식 주문 정말실망 입니다 8천원짜리 된장찌게가 더나을듯 합니다,0
3,1,한식이 아니라 그냥 식당밥입니다 사장님 불 친철...!,0
4,1,진짜 티비보고 절대가지마세요 ㅋㅋㅋㅋ4명이서 13만원 가까이 나왔는데 맛은 그냥 평...,0


In [6]:
df.to_csv("review_data.csv", index = False)

In [11]:
'''
TF-IDF를 이용한 핵심어 추출
 - 형태소 추출하기(한글 텍스트로 전처리)
'''
df = pd.read_csv("review_data.csv")

import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거
def text_cleaning(text):
    # 한글의 정규표현식으로 한글만 추출합니다.
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', str(text))
    return result

In [12]:
# 함수를 적용하여 리뷰에서 한글만 추출합니다.
df = pd.read_csv("review_data.csv")
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x))
del df['review']

# 한 글자 이상의 텍스트를 가지고 있는 데이터만 추출합니다.
df = df[df['ko_text'].str.len() > 0]
df.head()

,score,y,ko_text
0,3,0,맛은 있는데 가격이 아쉽
1,4,1,전 후기 잘 안쓰는게 실망했다는 리뷰가 많아 써봅니다 다른메뉴는 모르겠는데 이집은 ...
2,2,0,대실망입니다 양녕갈비살 정식 주문 정말실망 입니다 천원짜리 된장찌게가 더나을듯 합니다
3,1,0,한식이 아니라 그냥 식당밥입니다 사장님 불 친철
4,1,0,진짜 티비보고 절대가지마세요 ㅋㅋㅋㅋ명이서 만원 가까이 나왔는데 맛은 그냥 평범한 ...


In [13]:
'''형태소 단위로 추출'''

from konlpy.tag import Okt

# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word, tag) for word, tag in pos]
    return pos

# 형태소 추출 동작을 테스트 합니다.
result = get_pos(df['ko_text'].values[0])
print(result)

['맛/Noun', '은/Josa', '있는데/Adjective', '가격/Noun', '이/Josa', '아쉽/Adjective']
